# perfSONAR testpoint installation and initial configuration
<p>the following steps will install the `perfsonar-testpoint` bundle</p>
<p>depending on the goals for this node, additional tasks include:
<ul>
<li>configuring this perfSONAR (pS) node to participate in a mesh</li>
<li>setting up this node to participate in GridFTP disk-to-disk testing</p>
</ul>

## pull in the perfsonar-testpoint notebook
<p>using your terminal app log into your fionette as root, and cd into /root</p>
<p>use wget to transfer the perfsonar-testpoint notebook<pre>
wget http://fiona-10-01:8000/perfsonar-testpoint.ipynb -O perfsonar-testpoint.ipynb
</pre>
<p>launch jupyter lab</p>
<pre>jupyter lab --allow-root --ip=`your.fionette.ip`</pre>

## first we will prepare the system for installing the `perfsonar-testpoint` bundle

In [1]:
uname -r

3.10.0-693.17.1.el7.x86_64


In [2]:
sed -i 's/SELINUX=enforcing/SELINUX=disabled/g' /etc/selinux/config
setenforce 0

setenforce: SELinux is disabled


: 1

## setup NTP, bind-utils, traceroute (if not already present)

In [3]:
yum -y install ntp bind-utils traceroute

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
Package ntp-4.2.6p5-25.el7.centos.2.x86_64 already installed and latest version
Package 32:bind-utils-9.9.4-51.el7_4.2.x86_64 already installed and latest version
Package 3:traceroute-2.0.22-2.el7.x86_64 already installed and latest version
Nothing to do


In [4]:
systemctl enable ntpd

# adjust `/etc/ntp.conf` for the workshop NTP server
<p>review the contents of the file</p>

In [5]:
cat /etc/ntp.conf

# /etc/ntp.conf, configuration for NTP
# by default act only as a basic NTP client
restrict -4 default nomodify nopeer noquery notrap
restrict -6 default nomodify nopeer noquery notrap
# allow NTP messages from the loopback address, useful for debugging
restrict 127.0.0.1
restrict ::1
logfile /var/log/ntpd
driftfile /var/lib/ntp/ntp.drift
statsdir  /var/lib/ntp/
statistics loopstats peerstats clockstats
filegen loopstats file loopstats type day enable
filegen peerstats file peerstats type day enable
filegen clockstats file clockstats type day enable

# You should have at least 4 NTP servers

# server ntp-ucla.usno.navy.mil iburst   # Naval Observatory - Los Angeles, CA USA 

# server owamp.losa.net.internet2.edu iburst   # Internet2 - Los Angeles, CA USA 

# server owamp.salt.net.internet2.edu iburst   # Internet2 - Salt Lake City, UT USA 

# server saturn.es.net iburst   # ESnet - Sunnyvale, CA USA 

# server time3.chpc.utah.edu iburst   # UTAH - Server #3, University of Utah, Salt La

<p>using your favorite editor, comment out NTP servers outside of the workshop and add an entry for the local NTP server</p>
<pre>
#server time.apple.com
server 137.164.48.196 iburst
</pre>

## verify the changes

In [6]:
cat /etc/ntp.conf

# /etc/ntp.conf, configuration for NTP
# by default act only as a basic NTP client
restrict -4 default nomodify nopeer noquery notrap
restrict -6 default nomodify nopeer noquery notrap
# allow NTP messages from the loopback address, useful for debugging
restrict 127.0.0.1
restrict ::1
logfile /var/log/ntpd
driftfile /var/lib/ntp/ntp.drift
statsdir  /var/lib/ntp/
statistics loopstats peerstats clockstats
filegen loopstats file loopstats type day enable
filegen peerstats file peerstats type day enable
filegen clockstats file clockstats type day enable

# You should have at least 4 NTP servers

# server ntp-ucla.usno.navy.mil iburst   # Naval Observatory - Los Angeles, CA USA 

# server owamp.losa.net.internet2.edu iburst   # Internet2 - Los Angeles, CA USA 

# server owamp.salt.net.internet2.edu iburst   # Internet2 - Salt Lake City, UT USA 

# server saturn.es.net iburst   # ESnet - Sunnyvale, CA USA 

# server time3.chpc.utah.edu iburst   # UTAH - Server #3, University of Utah, Salt La

In [7]:
systemctl start ntpd

In [8]:
systemctl status ntpd -l

● ntpd.service - Network Time Service
   Loaded: loaded (/usr/lib/systemd/system/ntpd.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 15:09:02 PST; 16min ago
 Main PID: 1131 (ntpd)
   CGroup: /system.slice/ntpd.service
           └─1131 /usr/sbin/ntpd -u ntp:ntp -g

Mar 03 15:09:02 gigiot-22 systemd[1]: Starting Network Time Service...
Mar 03 15:09:02 gigiot-22 ntpd[1117]: ntpd 4.2.6p5@1.2349-o Wed Apr 12 21:24:06 UTC 2017 (1)
Mar 03 15:09:02 gigiot-22 ntpd[1131]: proto: precision = 0.100 usec
Mar 03 15:09:02 gigiot-22 ntpd[1131]: 0.0.0.0 c01d 0d kern kernel time sync enabled
Mar 03 15:09:02 gigiot-22 systemd[1]: Started Network Time Service.


In [9]:
ntpq -p -c rv

     remote           refid      st t when poll reach   delay   offset  jitter
*fiona-10-01     69.89.207.99     2 u   41   64  373    0.171  -10.920   1.729
associd=0 status=0614 leap_none, sync_ntp, 1 event, freq_mode,
version="ntpd 4.2.6p5@1.2349-o Wed Apr 12 21:24:06 UTC 2017 (1)",
processor="x86_64", system="Linux/3.10.0-693.17.1.el7.x86_64", leap=00,
stratum=3, precision=-23, rootdelay=67.828, rootdisp=56.109,
refid=137.164.48.196,
reftime=de45ac86.7c28226f  Sat, Mar  3 2018 15:21:42.484,
clock=de45ad78.3a570451  Sat, Mar  3 2018 15:25:44.227, peer=44920, tc=6,
mintc=3, offset=-24.116, frequency=-0.575, sys_jitter=0.000,
clk_jitter=9.107, clk_wander=0.203


## determine default interface

In [10]:
default_iface=$(awk '$2 == 00000000 { print $1 }' /proc/net/route)
echo $default_iface

enp3s0


## confirm `hostname -f` responds with FQHN

In [11]:
hostname -f

gigiot-22.conf.cenic.org


## use hostnamectl to set if its not a valid FQHN

In [12]:
myip=$(ip addr show dev $default_iface | grep 'inet '| awk '{print $2}' | awk -F "/" '{print $1}')
dig -x $myip +short

gigiot-22.


In [13]:
hostnamectl set-hostname `dig -x $myip +short`
hostname -f

gigiot-22.conf.cenic.org


## view current link settings on default interface

In [14]:
ip link show dev $default_iface
tc qdisc show

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000
    link/ether 40:8d:5c:f4:b6:c6 brd ff:ff:ff:ff:ff:ff
qdisc noqueue 0: dev lo root refcnt 2 
qdisc fq 0: dev enp3s0 root refcnt 2 limit 10000p flow_limit 100p buckets 1024 quantum 18028 initial_quantum 90140 


## create a custom udev rules file for tuning the NIC configuration 

In [15]:
if [ ! -f /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules ]; then
cat > /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules <<EOL
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k mtu 9000"
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k txqueuelen 10000"
EOL
chmod 644 /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules
fi

## Use udevadm to reload the rules and trigger the action

In [16]:
/sbin/udevadm control --reload-rules
/sbin/udevadm trigger

## confirm new NIC tuning settings are present

In [17]:
ip link show dev $default_iface
tc qdisc show

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000
    link/ether 40:8d:5c:f4:b6:c6 brd ff:ff:ff:ff:ff:ff
qdisc noqueue 0: dev lo root refcnt 2 
qdisc fq 0: dev enp3s0 root refcnt 2 limit 10000p flow_limit 100p buckets 1024 quantum 18028 initial_quantum 90140 


## Create a custom sysctl file with desired settings

In [18]:
if [ ! -f /etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf ]; then
cat>/etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf <<EOL
net.core.rmem_max = 67108864 
net.core.wmem_max = 67108864 
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control=htcp
EOL
fi
chmod 644 /etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf

## Adjust kernel boot-time defaults and rebuild grub.cfg

In [19]:
if ! grep --quiet default_hugepagesz /etc/default/grub; then
	sed -i 's/^GRUB_CMDLINE_LINUX=\"[^\"]*/& default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce/' /etc/default/grub
	grub2-mkconfig -o /boot/grub2/grub.cfg
fi

In [20]:
cat /etc/default/grub

GRUB_TIMEOUT=5
GRUB_DISTRIBUTOR="$(sed 's, release .*$,,g' /etc/system-release)"
GRUB_DEFAULT=saved
GRUB_DISABLE_SUBMENU=true
GRUB_TERMINAL_OUTPUT="console"
GRUB_CMDLINE_LINUX="crashkernel=auto rhgb quiet default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce"
GRUB_DISABLE_RECOVERY="true"


# SAVE your work NOW

In [ ]:
reboot

## confirm intended tuning settings are present 

In [1]:
ip link show | grep UP | grep -v lo
tc qdisc show

2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq state UP mode DEFAULT qlen 10000
qdisc noqueue 0: dev lo root refcnt 2 
qdisc fq 0: dev enp3s0 root refcnt 2 limit 10000p flow_limit 100p buckets 1024 quantum 18028 initial_quantum 90140 


## stop & disable a few un-wanted services and start & enable the `tuned` service

In [2]:
systemctl stop NetworkManager 
systemctl disable NetworkManager
systemctl stop irqbalance.service
systemctl disable irqbalance.service
systemctl stop wpa_supplicant
systemctl disable wpa_supplicant
systemctl start tuned
systemctl enable tuned


## check the current clock rate of the CPU cores

In [3]:
cat /proc/cpuinfo | grep Hz

model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000


## use `tuned-adm` to show the current active profile

In [4]:
tuned-adm active

Current active profile: throughput-performance


## set `network-throughput` at the active profile

In [5]:
tuned-adm profile network-throughput

## re-check the clock rate of the CPU cores

In [6]:
cat /proc/cpuinfo | grep Hz

model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000
model name	: Intel(R) Pentium(R) CPU  N3710  @ 1.60GHz
cpu MHz		: 1601.000


## Install the perfSONAR testpoint bundle (perfsonar-testpoint) and additional tools

<p>The assumption is that CentOS 7 is installed on your machine and that you have access to the necessary tools and configuration to install the perfsonar-testpoint software bundle.</p>
<p>First we will install some addditional and supplementary tools, if not already present on the system</p>

## Installing the perfSONAR testpoint bundle
<p>The instructions below are based on the perfSONAR CentOS Bundle Installation page, here:</p>
<p>http://docs.perfsonar.net/install_centos.html</p>
<p>with modifications for this workshop</p>
## First we need to install the Extra Packages for Enterprise Linux (EPEL)

## point yum to local perfSONAR repo 

In [7]:
cat << EOF > /etc/yum.repos.d/perfsonar.repo
[perfsonar]
name=Extra Packages for Enterprise Linux 7 - $basearch
baseurl=http://137.164.48.196/centos-7-x86_64/perfsonar
failovermethod=priority
enabled=1
gpgcheck=0
EOF

## Refresh the yum cache

In [8]:
yum clean all

Loaded plugins: fastestmirror
Cleaning repos: base epel extras globus perfsonar updates zfs
Cleaning up everything
Maybe you want: rm -rf /var/cache/yum, to also free up space taken by orphaned data from disabled or removed repos
Cleaning up list of fastest mirrors


## Now we can proceed to install the `perfsonar-testpoint` bundle:
<p>NOTE: due to the extensive output from installing this bundle, the call to yum includes the `-q` switch to set quiet mode.  output to `stderr` and any non-zero exit code will appear in the cell below.</p>
<p>NOTE: in a terminal session, run <pre>tail -f /var/log/yum.log</pre> to track the bundle installation</p>

In [9]:
yum -y -q install perfsonar-testpoint

Package perfsonar-testpoint-4.0.2.1-1.el7.centos.noarch already installed and latest version


## install optional `perfsonar-testpoint` packages

In [10]:
yum -y install perfsonar-toolkit-ntp perfsonar-toolkit-security perfsonar-toolkit-sysctl perfsonar-toolkit-servicewatcher perfsonar-toolkit-systemenv-testpoint

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
Package perfsonar-toolkit-ntp-4.0.2.2-1.el7.centos.noarch already installed and latest version
Package perfsonar-toolkit-security-4.0.2.2-1.el7.centos.noarch already installed and latest version
Package perfsonar-toolkit-sysctl-4.0.2.2-1.el7.centos.noarch already installed and latest version
Package perfsonar-toolkit-servicewatcher-4.0.2.2-1.el7.centos.noarch already installed and latest version
Package perfsonar-toolkit-systemenv-testpoint-4.0.2.2-1.el7.centos.noarch already installed and latest version
Nothing to do


## reload the firewall ruleset

In [11]:
firewall-cmd --reload

success


## check current active firewall rules

In [12]:
firewall-cmd --list-all

public (active)
  target: default
  icmp-block-inversion: no
  interfaces: enp3s0
  sources: 
  services: ssh dhcpv6-client ntp http https
  ports: 5001-5300/udp 5001-5300/tcp 5301-5600/udp 5301-5600/tcp 5601-5900/udp 5601-5900/tcp 6001-6200/udp 6001-6200/tcp 8760-9960/udp 8760-9960/tcp 8888/tcp 2811/tcp 7512/tcp 50000-51000/tcp 50000-51000/udp 2223/tcp 4823/tcp 5001-5900/tcp 5001-5900/udp
  protocols: 
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 
	


## NOTE: if the output of entry for `services:` entry does include `bwctl` or `owamp-control` re-run the script to configure the firewall rules for perfSONAR

In [13]:
/usr/lib/perfsonar/scripts/configure_firewall install

Adding perfSONAR firewall rules


## re-check firewall rules

In [14]:
firewall-cmd --list-all

public (active)
  target: default
  icmp-block-inversion: no
  interfaces: enp3s0
  sources: 
  services: ssh dhcpv6-client ntp http https traceroute bwctl owamp-control oppd
  ports: 5001-5300/udp 5001-5300/tcp 5301-5600/udp 5301-5600/tcp 5601-5900/udp 5601-5900/tcp 6001-6200/udp 6001-6200/tcp 8760-9960/udp 8760-9960/tcp 8888/tcp 2811/tcp 7512/tcp 50000-51000/tcp 50000-51000/udp 2223/tcp 4823/tcp 5001-5900/tcp 5001-5900/udp
  protocols: 
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 
	


## use `systemctl` to check the status of processes

In [15]:
systemctl status postgresql-9.5 -l
systemctl status httpd -l
systemctl status pscheduler* -l
systemctl status perfsonar-meshconfig-agent -l

● postgresql-9.5.service - PostgreSQL 9.5 database server
   Loaded: loaded (/usr/lib/systemd/system/postgresql-9.5.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 15:28:06 PST; 5min ago
 Main PID: 1923 (postgres)
   CGroup: /system.slice/postgresql-9.5.service
           ├─1923 /usr/pgsql-9.5/bin/postgres -D /var/lib/pgsql/9.5/data
           ├─1962 postgres: logger process                              
           ├─1971 postgres: checkpointer process                        
           ├─1972 postgres: writer process                              
           ├─1973 postgres: wal writer process                          
           ├─1974 postgres: autovacuum launcher process                 
           ├─1975 postgres: stats collector process                     
           ├─2444 postgres: pscheduler pscheduler 127.0.0.1(36250) idle 
           ├─2446 postgres: pscheduler pscheduler 127.0.0.1(36252) idle 
           ├─2447 postgres: pschedule

## Save your work NOW
## Reboot the server and verify everything came up as expected
## You will need to re-login to your machine and launch the notebook

In [ ]:
reboot

In [1]:
systemctl status postgresql-9.5 -l
systemctl status httpd -l
systemctl status pscheduler* -l
systemctl status perfsonar-meshconfig-agent -l

● postgresql-9.5.service - PostgreSQL 9.5 database server
   Loaded: loaded (/usr/lib/systemd/system/postgresql-9.5.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 15:34:52 PST; 1min 10s ago
  Process: 1870 ExecStart=/usr/pgsql-9.5/bin/pg_ctl start -D ${PGDATA} -s -w -t 300 (code=exited, status=0/SUCCESS)
  Process: 1839 ExecStartPre=/usr/pgsql-9.5/bin/postgresql95-check-db-dir ${PGDATA} (code=exited, status=0/SUCCESS)
 Main PID: 1898 (postgres)
   CGroup: /system.slice/postgresql-9.5.service
           ├─1898 /usr/pgsql-9.5/bin/postgres -D /var/lib/pgsql/9.5/data
           ├─1954 postgres: logger process                              
           ├─1967 postgres: checkpointer process                        
           ├─1968 postgres: writer process                              
           ├─1969 postgres: wal writer process                          
           ├─1970 postgres: autovacuum launcher process                 
           ├─1971 po

## update the `meshconfig-agent` to add a reference for the group mesh json
<p>review the current contents</p>

In [2]:
cat /etc/perfsonar/meshconfig-agent.conf

## Use 'mesh' blocks to to specify each mesh that the agent should
## configure the host against
#<mesh>
#    ## Use 'configuration_url' to specify the URL where the agent should obtain
#    ## the mesh configuration from
#    configuration_url             https://host.domain.edu/example.json   
#
#    ## To ensure that the configuration is trusted, you can set the
#    ## 'validate_certificate' option to 1. This will validate that the certificate
#    ## is valid, and matches the hostname. If the 'validate_certificate' option is
#    ## set to 1, the 'ca_certificate_file' option must be set.
#    #validate_certificate         0
#
#    ## The 'ca_certificate_file' specifies which CAs to use to validate the
#    ## certificates.
#    #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt    # the default RedHat 6 CAs
#
#    ## The 'required' field specifies that if this mesh isn't available, or has
#    ## problems, to skip making any changes to the host configuration.
#    #req

## update the `meshconfig-agent.conf` with a mesh stanza referencing the group mesh json

In [1]:
cd /etc/perfsonar
wget http://fiona-10-01:8000/host-to-group -O /usr/local/etc/prp-fiona-ws--host-to-group
mygrp=$(grep $(hostname -s) /usr/local/etc/prp-fiona-ws--host-to-group | awk '{print $4}')
if [ ! -f /etc/perfsonar/meshconfig-agent.conf.orig ]; then
cp /etc/perfsonar/meshconfig-agent.conf /etc/perfsonar/meshconfig-agent.conf.orig
fi
cat >> /etc/perfsonar/meshconfig-agent.conf <<EOL
<mesh>
   configuration_url http://gigiot-ma-$mygrp/prp-fiona-ws-mesh--gigiot-ma-$mygrp.json
   #validate_certificate         0
   #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
   configure_archives 1
</mesh>
EOL
chown perfsonar:perfsonar /etc/perfsonar/meshconfig-agent.conf
chmod 644 /etc/perfsonar/meshconfig-agent.conf

--2018-03-04 00:22:49--  http://fiona-10-01:8000/host-to-group
Resolving fiona-10-01 (fiona-10-01)... 137.164.48.196
Connecting to fiona-10-01 (fiona-10-01)|137.164.48.196|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 859 [application/octet-stream]
Saving to: ‘/usr/local/etc/prp-fiona-ws--host-to-group’

100%[======================================>] 859         --.-K/s   in 0s      

2018-03-04 00:22:49 (51.6 MB/s) - ‘/usr/local/etc/prp-fiona-ws--host-to-group’ saved [859/859]



In [2]:
cat /etc/perfsonar/meshconfig-agent.conf

## Use 'mesh' blocks to to specify each mesh that the agent should
## configure the host against
#<mesh>
#    ## Use 'configuration_url' to specify the URL where the agent should obtain
#    ## the mesh configuration from
#    configuration_url             https://host.domain.edu/example.json   
#
#    ## To ensure that the configuration is trusted, you can set the
#    ## 'validate_certificate' option to 1. This will validate that the certificate
#    ## is valid, and matches the hostname. If the 'validate_certificate' option is
#    ## set to 1, the 'ca_certificate_file' option must be set.
#    #validate_certificate         0
#
#    ## The 'ca_certificate_file' specifies which CAs to use to validate the
#    ## certificates.
#    #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt    # the default RedHat 6 CAs
#
#    ## The 'required' field specifies that if this mesh isn't available, or has
#    ## problems, to skip making any changes to the host configuration.
#    #req

## restart the `perfsonar-meshconfig-agent` process

In [3]:
systemctl restart perfsonar-meshconfig-agent

## verify the contents of `meshconfig-agent-tasks.conf` to determine if the mesh json has been processed correctly
<p>it should have `<test>` stanzas which includes sections for test schedule, parameters, targets, and measurement archives</p> 

In [4]:
cat /etc/perfsonar/meshconfig-agent-tasks.conf

<test>
    added_by_mesh   1
    description   iperf3 Throughput, disjoint
    <schedule>
        random_start_percentage   25
        type   regular_intervals
        interval   7200
    </schedule>
    <parameters>
        tool   iperf3
        omit_interval   5
        type   bwctl
        duration   30
        force_ipv4   1
    </parameters>
    target   gigiot-06
    target   gigiot-07
    target   gigiot-13
    target   gigiot-14
    target   gigiot-20
    target   gigiot-21
    local_address   gigiot-22
    <measurement_archive>
        database   http://137.164.48.232/esmond/perfsonar/archive/
        added_by_mesh   1
        type   esmond/throughput
    </measurement_archive>
    <created_by>
        name   Earth Sciences perfSONAR
        agent_type   remote-mesh
        uri   http://gigiot-ma-4/prp-fiona-ws-mesh--gigiot-ma-4.json
    </created_by>
</test>
<test>
    added_by_mesh   1
    description   iperf3 Throughput
    <schedule>
        random_start_percentage   25
  

# confirming a succesful installation and mesh participation


## open a terminal session and run the following to help determine the functionality of the system
<ul>
<li> `pscheduler monitor` to show an updating display of completed, running, and pending tasks</li>
<li> `pscheduler task throughput --dest some.other.ps.node` to run a throughput (iperf3) test</li>
<li> `pscheduler task latency --dest some.other.ps.node` to run a latency/loss (owping) test</li> 
</ul>

# perfSONAR testpoint appendix: perfSONAR testpoint installation troubleshooting 

## pscheduler processes fail to start 
<p>there are reports of a recent dependency breakage in the perfSONAR repo which may cause `httpd` to be absent from the bundle installation. pScheduler will not function without httpd.  if you attempt to use pscheduler and you receive an error message that says:<pre>Unable to find pScheduler on localhost</pre> this can be an indication that httpd is missing.</p>
<p>install and enable httpd if not present on the system</p>

In [5]:
yum -y -q install httpd
systemctl enable httpd
systemctl start httpd
systemctl status httpd -l

Package httpd-2.4.6-67.el7.centos.6.x86_64 already installed and latest version
● httpd.service - The Apache HTTP Server
   Loaded: loaded (/usr/lib/systemd/system/httpd.service; enabled; vendor preset: disabled)
   Active: active (running) since Sat 2018-03-03 19:43:58 PST; 4h 44min ago
     Docs: man:httpd(8)
           man:apachectl(8)
 Main PID: 1892 (httpd)
   Status: "Total requests: 8005; Current requests/sec: 0; Current traffic:   0 B/sec"
   CGroup: /system.slice/httpd.service
           ├─1892 /usr/sbin/httpd -DFOREGROUND
           ├─2055 /usr/sbin/httpd -DFOREGROUND
           ├─2056 /usr/sbin/httpd -DFOREGROUND
           ├─2057 /usr/sbin/httpd -DFOREGROUND
           ├─2058 /usr/sbin/httpd -DFOREGROUND
           ├─2059 /usr/sbin/httpd -DFOREGROUND
           ├─2060 /usr/sbin/httpd -DFOREGROUND
           ├─2061 /usr/sbin/httpd -DFOREGROUND
           ├─2062 /usr/sbin/httpd -DFOREGROUND
           ├─2065 /usr/sbin/httpd -DFOREGROUND
           ├─2066 /usr/sbin/httpd -DFOR

## python library incompatibility on systems having both perSONAR and GridFTP
<p>We have seen an issue with the GridFTP installation where an incompatible python library is installed and this causes the pscheduler processes to not start. If your pscheduler-* processes do not start you can try the following:</p>

In [ ]:
pip uninstall urllib3
yum install -y python-urllib3
systemctl start pscheduler-*
systemctl status pscheduler-* -l

# Setting up scheduled GridFTP testing
<p>the following steps will set up cron jobs to accomplish</p>
<ul>
<li>a schedule of GridFTP transfers from a group of participating nodes</li>
<li>parsing the GridFTP transfer log and uploading the results to a central Measurement Archive (MA)</li>
</ul>

## wget the host-to-group map and the group-specific script for GridFTP and make it executable

In [6]:
wget http://fiona-10-01:8000/host-to-group -O /usr/local/etc/prp-fiona-ws--host-to-group
mygrp=$(grep $(hostname -s) /usr/local/etc/prp-fiona-ws--host-to-group | awk '{print $4}')
wget http://fiona-10-01:8000/cron-load-gridftp-grp-0$mygrp.sh -O /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh
chmod 755 /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh

--2018-03-04 00:29:19--  http://fiona-10-01:8000/host-to-group
Resolving fiona-10-01 (fiona-10-01)... 137.164.48.196
Connecting to fiona-10-01 (fiona-10-01)|137.164.48.196|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 859 [application/octet-stream]
Saving to: ‘/usr/local/etc/prp-fiona-ws--host-to-group’

100%[======================================>] 859         --.-K/s   in 0s      

2018-03-04 00:29:19 (85.3 MB/s) - ‘/usr/local/etc/prp-fiona-ws--host-to-group’ saved [859/859]

--2018-03-04 00:29:19--  http://fiona-10-01:8000/cron-load-gridftp-grp-04.sh
Resolving fiona-10-01 (fiona-10-01)... 137.164.48.196
Connecting to fiona-10-01 (fiona-10-01)|137.164.48.196|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 905 [application/x-sh]
Saving to: ‘/usr/local/bin/cron-load-gridftp-grp-04.sh’

100%[======================================>] 905         --.-K/s   in 0s      

2018-03-04 00:29:19 (66.3 MB/s) - ‘/usr/local/bin/cron-load-gridft

## inspect the contents of the script

In [7]:
cat /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh

#!/bin/bash
TMPFILE=`mktemp`
HEAD_NODE=137.164.48.196:8000
MA_NODE=gigiot-ma-4
GRP=04
MY_HOSTNAME_SHORT=`hostname -s`
GRIDFTP_API_KEY=

python /usr/bin/esmond-ps-load-gridftp -p /opt/esmond-gridftp/prp-fiona-ws-grp-$GRP.pickle -U http://$MA_NODE/esmond -u gridftp -k $GRIDFTP_API_KEY -f /var/log/gridftp-transfer.log

wget http://$HEAD_NODE/cron-gridftp-transfer-mesh-grp-$GRP.sh -O /usr/local/bin/cron-gridftp-transfer-mesh-grp-$GRP.sh
chmod 755 /usr/local/bin/cron-gridftp-transfer-mesh-grp-$GRP.sh
sed -i '/$MY_HOSTNAME_SHORT/s/^/#/' /usr/local/bin/cron-gridftp-transfer-mesh-grp-$GRP.sh
curl -s  http://$HEAD_NODE/cron-mesh-timing-grp-$GRP.sh -o $TMPFILE

grep -iq $MY_HOSTNAME_SHORT $TMPFILE
RC=$?

if [[ $RC -eq 0 ]];then
      grep -i $MY_HOSTNAME_SHORT $TMPFILE > /etc/cron.d/cron-gridftp-transfer-mesh-grp-$GRP
else
      logger -t "$0" 'Error while downloading cronjob files'
fi

rm -f $TMPFILE


## using your favorite editor, update the `GRIDFTP_API_KEY` entry
<p>update the <pre>GRIDFTP_API_KEY</pre> entry with the API key generated earlier for the gridftp user on the MA for your group<p>

## Grab a helper script that gragefully times out the globus url copy if it exceeds a value in seconds.

In [8]:
mkdir /opt/esmond-gridftp/
wget http://fiona-10-01:8000/timeout.sh -O /usr/local/bin/timeout.sh
chmod 755 /usr/local/bin/timeout.sh

mkdir: cannot create directory ‘/opt/esmond-gridftp/’: File exists
--2018-03-04 00:31:58--  http://fiona-10-01:8000/timeout.sh
Resolving fiona-10-01 (fiona-10-01)... 137.164.48.196
Connecting to fiona-10-01 (fiona-10-01)|137.164.48.196|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2658 (2.6K) [application/x-sh]
Saving to: ‘/usr/local/bin/timeout.sh’

100%[======================================>] 2,658       --.-K/s   in 0s      

2018-03-04 00:31:58 (109 MB/s) - ‘/usr/local/bin/timeout.sh’ saved [2658/2658]



## This script scrapes the gridftp transfer log for transfers that it has not yet processed
<p>The results are uploaded to the central measurement archive running on the MA host for your group.</p>

In [9]:
/usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh 

ts=2018-03-04 00:32:16,263 event=main.start id=1520152336 found last record: {'code': '226', 'date': '20180304071916.834642', 'file': '/data/1G.dat', 'start': '20180304071831.403478', 'retrans': '0', 'nl_evnt': 'FTP_INFO', 'stripes': '1', 'volume': '/', 'dest': '[137.164.48.217]', 'host': 'gigiot-22.conf.cenic.org', 'buffer': '235104', 'user': 'anonymous', 'taskid': 'none', 'prog': 'globus-gridftp-server', 'streams': '1', 'type': 'RETR', 'block': '262144', 'nbytes': '1073741824'}
ts=2018-03-04 00:32:16,270 event=main.process id=1520152336 File /var/log/gridftp-transfer.log passes last record check
ts=2018-03-04 00:32:17,422 event=scan_and_load_netlogger.end id=1520152337 Loaded 8 records
--2018-03-04 00:32:17--  http://137.164.48.196:8000/cron-gridftp-transfer-mesh-grp-04.sh
Connecting to 137.164.48.196:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3509 (3.4K) [application/x-sh]
Saving to: ‘/usr/local/bin/cron-gridftp-transfer-mesh-grp-04.sh’

100%[=========


## Running this file the first time pulls in the cron-gridftp-transfer-mesh for this host

In [10]:
chmod 644 /etc/cron.d/cron-gridftp-transfer-mesh-grp-0$mygrp

## Inspect the cron job

In [11]:
cat /etc/cron.d/cron-gridftp-transfer-mesh-grp-0$mygrp

15 6,8,10,12,14,16,18,20,22,24,2,4 * * * root /usr/local/bin/cron-gridftp-transfer-mesh-grp-04.sh &> /var/log/cron-gridftp-transfer-mesh-grp-04.log # gigiot-22
31 * * * * root /usr/local/bin/cron-load-gridftp-grp-04.sh &> /var/log/cron-load-gridftp-grp-04.log # gigiot-22
